#0. Prepare environment

In [8]:
import torch

print("PyTorch version: ", torch.__version__)
print("CUDA version: ", torch.version.cuda)
print("cuDNN version: ", torch.backends.cudnn.version())
print("Is CUDA available: ", torch.cuda.is_available())
print("Is cuDNN available: ", torch.backends.cudnn.is_available())
print(torch.cuda.device_count())

PyTorch version:  1.12.1+cu113
CUDA version:  11.3
cuDNN version:  8302
Is CUDA available:  True
Is cuDNN available:  True
1


In [19]:
!pip install -r requirements_lora.txt

  Using cached datasets-2.13.1-py3-none-any.whl (486 kB)
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached xxhash-3.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (213 kB)
  Using cached aiohttp-3.8.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached tensorboard_data_server-0.7.1-py3-none-manylinux2014_x86_64.whl (6.6 MB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
  Using cached google_auth-2.22.0-py2.py3-none-any.whl (181 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached grpcio-1.56.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
  Using cached yarl-1.9.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (266 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.3.3-cp38-cp38-manylinux_2_5_x86_64.man

In [4]:
MODEL_NAME="runwayml/stable-diffusion-v1-5"
TRAIN_DIR="generated_data"
OUTPUT_DIR="lora_model"

#1. Train with LoRA

In [24]:
!accelerate launch --config_file=config.yaml \
  --mixed_precision="fp16" \
  train_text_to_image_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=10000 \
  --learning_rate=1e-04 \
  --max_grad_norm=1 \
  --lr_scheduler="cosine" --lr_warmup_steps=0 \
  --output_dir=$OUTPUT_DIR \
  --checkpointing_steps=500 \
  --validation_prompt="draw, airplane" \
  --seed=1337

07/11/2023 21:54:58 - INFO - __main__ - Distributed environment: MULTI_GPU  Backend: nccl
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda:0

Mixed precision type: fp16

{'thresholding', 'clip_sample_range', 'dynamic_thresholding_ratio', 'variance_type', 'sample_max_value', 'timestep_spacing', 'prediction_type'} was not found in config. Values will be initialized to default values.
{'scaling_factor'} was not found in config. Values will be initialized to default values.
{'num_attention_heads', 'conv_in_kernel', 'mid_block_type', 'num_class_embeds', 'timestep_post_act', 'addition_embed_type_num_heads', 'mid_block_only_cross_attention', 'upcast_attention', 'class_embed_type', 'time_embedding_dim', 'projection_class_embeddings_input_dim', 'transformer_layers_per_block', 'class_embeddings_concat', 'addition_embed_type', 'encoder_hid_dim_type', 'dual_cross_attention', 'cross_attention_norm', 'resnet_out_scale_factor', 'conv_out_kernel', 'resnet_time_scale_shift', 'time_

In [6]:
!pip install transformers --upgrade
!pip install diffusers
!pip install accelerate


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


#2. Inference

In [25]:
import torch
from diffusers import StableDiffusionPipeline

model_base=MODEL_NAME
pipe = StableDiffusionPipeline.from_pretrained(model_base, torch_dtype=torch.float16)

lora_model_path = OUTPUT_DIR

pipe.unet.load_attn_procs(lora_model_path)
pipe.to("cuda")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.18.2",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [31]:
prompt = "draw, toy"
image = pipe(prompt, num_inference_steps=25, guidance_scale=7.5).images[0]
image.save(prompt + ".png")

100%|██████████| 25/25 [00:05<00:00,  4.69it/s]
